<a href="https://colab.research.google.com/github/eltondeoliveirasilva/Hiperparametros./blob/main/Hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.datasets import make_moons
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor

In [20]:
features = pd.read_excel('/content/drive/MyDrive/temps.xlsx')
features.head()

,year,month,day,week,temp_2,temp_1,average,actual
0,2016,1,1,Fri,45,45,45.6,45
1,2016,1,2,Sat,44,45,45.7,44
2,2016,1,3,Sun,45,44,45.8,41
3,2016,1,4,Mon,44,41,45.9,40
4,2016,1,5,Tues,41,40,46.0,44


In [21]:
features.describe()

,year,month,day,temp_2,temp_1,average,actual
count,348.0,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000
mean,2016.0,6.477011,15.514368,62.652299,62.701149,59.760632,62.543103
std,0.0,3.498380,8.772982,12.165398,12.120542,10.527306,11.794146
min,2016.0,1.000000,1.000000,35.000000,35.000000,45.100000,35.000000
25%,2016.0,3.000000,8.000000,54.000000,54.000000,49.975000,54.000000
50%,2016.0,6.000000,15.000000,62.500000,62.500000,58.200000,62.500000
75%,2016.0,10.000000,23.000000,71.000000,71.000000,69.025000,71.000000
max,2016.0,12.000000,31.000000,117.000000,117.000000,77.400000,92.000000


In [22]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   year     348 non-null    int64  
 1   month    348 non-null    int64  
 2   day      348 non-null    int64  
 3   week     348 non-null    object 
 4   temp_2   348 non-null    int64  
 5   temp_1   348 non-null    int64  
 6   average  348 non-null    float64
 7   actual   348 non-null    int64  
dtypes: float64(1), int64(6), object(1)
memory usage: 21.9+ KB


In [23]:
features = pd.get_dummies(features)
features.head()

,year,month,day,temp_2,temp_1,average,actual,week_Fri,week_Mon,week_Sat,week_Sun,week_Thurs,week_Tues,week_Wed
0,2016,1,1,45,45,45.6,45,True,False,False,False,False,False,False
1,2016,1,2,44,45,45.7,44,False,False,True,False,False,False,False
2,2016,1,3,45,44,45.8,41,False,False,False,True,False,False,False
3,2016,1,4,44,41,45.9,40,False,True,False,False,False,False,False
4,2016,1,5,41,40,46.0,44,False,False,False,False,False,True,False


In [24]:
labels = np.array(features['actual'])

features= features.drop('actual', axis = 1)

feature_list = list(features.columns)

features = np.array(features)

In [25]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [28]:
gbr = GradientBoostingRegressor()
gbr.fit(train_features, train_labels)

GradientBoostingRegressor()

In [31]:
gbr_pred = gbr.predict(test_features)

In [32]:
r_sq = gbr.score(features, labels)
print('R^2:',r_sq)
print('MAE:', metrics.mean_absolute_error(test_labels, gbr_pred))
print('MSE:', metrics.mean_squared_error(test_labels, gbr_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(test_labels, gbr_pred)))

R^2: 0.9175946430769122
MAE: 4.074160201320348
MSE: 28.507726085457673
RMSE: 5.339262691182901


In [33]:
# Hiperparametro

In [39]:
from sklearn.model_selection import GridSearchCV

parameters = {'learning_rate': [0.03],
              'subsample'    : [ 0.2],
              'n_estimators' : [100],
              'max_depth'    : [8]
}


In [40]:
grid_search = GridSearchCV(gbr, parameters,scoring='r2', cv = 2, n_jobs=-1)
grid_search.fit(train_features, train_labels)

GridSearchCV(cv=2, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.03], 'max_depth': [8],
                         'n_estimators': [100], 'subsample': [0.2]},
             scoring='r2')

In [38]:
print('resultados do GridSearch')
print('Melhores estimadores:', grid_search.best_estimator_)
print('Melhores Parametros:', grid_search.best_params_)

resultados do GridSearch
Melhores estimadores: GradientBoostingRegressor(learning_rate=0.03, max_depth=8, subsample=0.2)
Melhores Parametros: {'learning_rate': 0.03, 'max_depth': 8, 'n_estimators': 100, 'subsample': 0.2}


In [41]:
best_model = grid_search.best_estimator_

In [42]:
best_model.get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.03,
 'loss': 'squared_error',
 'max_depth': 8,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 0.2,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [45]:
gbr_tunned = GradientBoostingRegressor(alpha= 0.9,
 ccp_alpha = 0.0,
 criterion = 'friedman_mse',
 init =  None,
 learning_rate = 0.03,
 loss = 'squared_error',
 max_depth = 8,
 max_features = None,
 max_leaf_nodes = None,
 min_impurity_decrease = 0.0,
 min_samples_leaf = 1,
 min_samples_split = 2,
 min_weight_fraction_leaf = 0.0,
 n_estimators = 100,
 n_iter_no_change = None,
 random_state = None,
 subsample = 0.2,
 tol =0.0001,
 validation_fraction = 0.1,
 verbose = 0,
 warm_start = False)

In [47]:
gbr_tunned.fit(train_features, train_labels)
gbr_tunned_pred = gbr_tunned.predict(test_features)

In [49]:
r_sq = gbr_tunned.score(features, labels)
print('R^2:',r_sq)
print('MAE:', metrics.mean_absolute_error(test_labels, gbr_tunned_pred))

R^2: 0.9050874340466327
MAE: 3.708645338801353
